# IR Lab SoSe 2024: Erweitertes Retrieval System mit gewichtetem TF-IDF

Dieses Jupyter Notebook implementiert ein erweitertes Retrieval-System, das TF-IDF verwendet und zusätzlich die einzelnen Felder (Titel und Abstract) gewichtet.
Wir verwenden ein Korpus wissenschaftlicher Arbeiten (Titel + Abstracts) aus den Bereichen Information Retrieval und Natural Language Processing.

### Schritt 1: Bibliotheken importieren

In [1]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from pyterrier.pipelines import *

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Erstellen Sie einen REST-Client für die TIRA-Plattform
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


### Schritt 2: Dataset und Index laden

In [3]:
# Das Dataset: die Vereinigung der IR Anthology und der ACL Anthology
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# Ein (vorgebauter) PyTerrier-Index, geladen von TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

### Schritt 3: Erweiterte Retrieval Pipeline mit gewichtetem TF-IDF definieren

In [4]:
# Definieren Sie die Felder und ihre Gewichte
fields = ["title", "abstract"]
weights = [2.0, 1.0]  # Titel hat doppeltes Gewicht des Abstracts

# Erstellen Sie ein gewichtetes TF-IDF-Modell
weighted_tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", controls={"c": 1.0}, fields=fields)

# Wenden Sie die Feldgewichte an
for field, weight in zip(fields, weights):
    weighted_tfidf.controls[f"w.{field}"] = weight

# Kombinieren Sie BM25 und gewichtetes TF-IDF
combined_pipeline = pt.BatchRetrieve(index, wmodel="BM25") >> weighted_tfidf

### Schritt 4: Retrieval durchführen

In [5]:
print('Zunächst werfen wir einen kurzen Blick auf die ersten drei Topics:')
pt_dataset.get_topics('text').head(3)

Zunächst werfen wir einen kurzen Blick auf die ersten drei Topics:


,qid,query
0,1,retrieval system improving effectiveness
1,2,machine learning language identification
2,3,social media detect self harm


In [6]:
print('Jetzt führen wir das Retrieval durch...')
run = combined_pipeline(pt_dataset.get_topics('text'))

print('Fertig. Hier sind die ersten 10 Einträge des Runs')
run.head(10)

Jetzt führen wir das Retrieval durch...


BR(TF_IDF): 100%|██████████| 68/68 [00:01<00:00, 57.09q/s]


Fertig. Hier sind die ersten 10 Einträge des Runs


,qid,docid,docno,rank,score,query
0,1,125137,1989.ipm_journal-ir0volumeA25A4.2,0,12.493450,retrieval system improving effectiveness
1,1,94858,2004.cikm_conference-2004.47,1,10.246570,retrieval system improving effectiveness
2,1,111672,2003.trec_conference-2003.21,2,10.035867,retrieval system improving effectiveness
3,1,123051,2002.ipm_journal-ir0volumeA38A1.0,3,10.035867,retrieval system improving effectiveness
4,1,110990,1998.trec_conference-1998.34,4,9.866787,retrieval system improving effectiveness
5,1,53327,O07-2010,5,9.545162,retrieval system improving effectiveness
6,1,78846,2016.iir_workshop-2016.13,6,9.545162,retrieval system improving effectiveness
7,1,85224,2010.riao_conference-2010.33,7,9.545162,retrieval system improving effectiveness
8,1,90655,2017.airs_conference-2017.1,8,9.545162,retrieval system improving effectiveness
9,1,91391,2004.ecir_conference-2004.21,9,9.545162,retrieval system improving effectiveness


### Schritt 5: Run-Datei für nachfolgende Auswertungen persistieren

In [7]:
persist_and_normalize_run(run, system_name='weighted-tfidf-combined', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".


### Schritt 6: Analyse und Vergleich

Um die Auswirkungen der Feldgewichtung zu verstehen, können Sie die Ergebnisse dieses Runs mit denen des ursprünglichen TF-IDF-Ansatzes vergleichen. Achten Sie besonders auf Änderungen in der Reihenfolge der Dokumente und auf mögliche Verbesserungen bei der Relevanz der Top-Ergebnisse.

Für eine gründliche Evaluation sollten Sie Metriken wie MAP (Mean Average Precision), NDCG (Normalized Discounted Cumulative Gain) oder Precision@k verwenden. Dies erfordert jedoch ein separates Evaluationsnotebook mit Zugriff auf Relevanzurteile.